<a href="https://colab.research.google.com/github/thaiphi/Stock-Markets-Analytics-2024/blob/main/Module_3_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this homework, we're going to work with categorical variables, first ML models (Decision Trees), and hyperparameter tuning.

If not stated otherwise, please use the Colab Module 3 covered in the livestream to re-use the code snippets.

Data: the new version of the data (after all transformations) is here: tbd

In [1]:

!pip install yfinance

In [2]:
# read files shared via google-drive-link
# https://stackoverflow.com/questions/62759748/downloading-data-from-a-shared-google-drive-link-in-google-colab

!pip uninstall gdown -y && pip install gdown
!gdown -V

Found existing installation: gdown 5.1.0
Uninstalling gdown-5.1.0:
  Successfully uninstalled gdown-5.1.0
gdown 5.2.0 at /usr/local/lib/python3.10/dist-packages


In [3]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.graph_objects as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt


#Question 1 (1 point): Dummies on Month and Week-of-Month
Find the CORRELATION VALUE of the most correlated dummy <month-week_of_month> with the binary outcome variable (" is_positive_growth_5d_future")?

You saw in the correlation analysis and modeling that September and October may be important seasonal months. In this task, we'll go futher and try to generate dummies for Month and Week-of-month (starting from 1). For example, the first week of October should be coded similar to this: 'October_w1'. Once you've generated the new set of variables, find the most correlated (in absolute value) one with "is_positive_growth_5d_future" and truncate it to 3 digits after the comma.

Suggested path to a solution:

1. [Source] Use this formula to get the week of month for the datetime variable d: (d.day-1)//7+1

2. Generate string values for each month-week_of_month combination

3. Use pandas.get_dummies() to generate dummies

4. Use pandas.DataFrame.corr() function (also used in [Code Snippet 1]) to get correlations with "is_positive_growth_5d_future", filter out only variables representing the new dummy, and sort it by abs. values (you can define a new column in the dataframe with correlations), and find the highest value (among new dummies set).

In [4]:
# https://stackoverflow.com/questions/62759748/downloading-data-from-a-shared-google-drive-link-in-google-colab
# truncated data from Module 2: https://drive.google.com/file/d/1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb/view?usp=sharing
!gdown https://drive.google.com/file/d/1kNWWPi49td0EZhmi6LzNCa2ssC5IUxHP/view?usp=sharing --fuzzy -O /content/

Downloading...
From (original): https://drive.google.com/uc?id=1kNWWPi49td0EZhmi6LzNCa2ssC5IUxHP
From (redirected): https://drive.google.com/uc?id=1kNWWPi49td0EZhmi6LzNCa2ssC5IUxHP&confirm=t&uuid=3c36cd71-329b-4b23-a2ed-813a0a074934
To: /content/stocks_df_combined_2024_05_07.parquet.brotli
100% 119M/119M [00:00<00:00, 144MB/s]


In [5]:
# truncated
# df = pd.read_parquet("/content/stocks_df_combined_trunc_2014_2023.parquet.brotli", )

# full dataset for 33 stocks
df_full = pd.read_parquet("/content/stocks_df_combined_2024_05_07.parquet.brotli", )

In [6]:
df_full.keys()

Index(['Open', 'High', 'Low', 'Close', 'Adj Close_x', 'Volume', 'Ticker',
       'Year', 'Month', 'Weekday',
       ...
       'growth_brent_oil_7d', 'growth_brent_oil_30d', 'growth_brent_oil_90d',
       'growth_brent_oil_365d', 'growth_btc_usd_1d', 'growth_btc_usd_3d',
       'growth_btc_usd_7d', 'growth_btc_usd_30d', 'growth_btc_usd_90d',
       'growth_btc_usd_365d'],
      dtype='object', length=202)

In [8]:
# growth indicators (but not future growth)
GROWTH = [g for g in df_full.keys() if (g.find('growth_')==0)&(g.find('future')<0)]
GROWTH

['growth_1d',
 'growth_3d',
 'growth_7d',
 'growth_30d',
 'growth_90d',
 'growth_365d',
 'growth_dax_1d',
 'growth_dax_3d',
 'growth_dax_7d',
 'growth_dax_30d',
 'growth_dax_90d',
 'growth_dax_365d',
 'growth_snp500_1d',
 'growth_snp500_3d',
 'growth_snp500_7d',
 'growth_snp500_30d',
 'growth_snp500_90d',
 'growth_snp500_365d',
 'growth_dji_1d',
 'growth_dji_3d',
 'growth_dji_7d',
 'growth_dji_30d',
 'growth_dji_90d',
 'growth_dji_365d',
 'growth_epi_1d',
 'growth_epi_3d',
 'growth_epi_7d',
 'growth_epi_30d',
 'growth_epi_90d',
 'growth_epi_365d',
 'growth_gold_1d',
 'growth_gold_3d',
 'growth_gold_7d',
 'growth_gold_30d',
 'growth_gold_90d',
 'growth_gold_365d',
 'growth_wti_oil_1d',
 'growth_wti_oil_3d',
 'growth_wti_oil_7d',
 'growth_wti_oil_30d',
 'growth_wti_oil_90d',
 'growth_wti_oil_365d',
 'growth_brent_oil_1d',
 'growth_brent_oil_3d',
 'growth_brent_oil_7d',
 'growth_brent_oil_30d',
 'growth_brent_oil_90d',
 'growth_brent_oil_365d',
 'growth_btc_usd_1d',
 'growth_btc_usd_3d',


In [9]:
# leaving only Volume ==> generate ln(Volume)
OHLCV = ['Open','High','Low','Close','Adj Close_x','Volume']
OHLCV

['Open', 'High', 'Low', 'Close', 'Adj Close_x', 'Volume']

In [10]:
CATEGORICAL = ['Month', 'Weekday', 'Ticker', 'ticker_type']

In [11]:
TO_PREDICT = [g for g in df_full.keys() if (g.find('future')>=0)]
TO_PREDICT

['growth_future_5d', 'is_positive_growth_5d_future']

In [12]:
TO_DROP = ['Year','Date','index_x', 'index_y', 'index', 'Quarter','Adj Close_y'] + CATEGORICAL + OHLCV
TO_DROP


['Year',
 'Date',
 'index_x',
 'index_y',
 'index',
 'Quarter',
 'Adj Close_y',
 'Month',
 'Weekday',
 'Ticker',
 'ticker_type',
 'Open',
 'High',
 'Low',
 'Close',
 'Adj Close_x',
 'Volume']

In [13]:
# let's define on more custom numerical features
df_full['ln_volume'] = df_full.Volume.apply(lambda x: np.log(x))

In [14]:
# manually defined features
CUSTOM_NUMERICAL = ['SMA10', 'SMA20', 'growing_moving_average', 'high_minus_low_relative','volatility', 'ln_volume']

In [15]:
# All Supported Ta-lib indicators: https://github.com/TA-Lib/ta-lib-python/blob/master/docs/funcs.md

TECHNICAL_INDICATORS = ['adx', 'adxr', 'apo', 'aroon_1','aroon_2', 'aroonosc',
 'bop', 'cci', 'cmo','dx', 'macd', 'macdsignal', 'macdhist', 'macd_ext',
 'macdsignal_ext', 'macdhist_ext', 'macd_fix', 'macdsignal_fix',
 'macdhist_fix', 'mfi', 'minus_di', 'mom', 'plus_di', 'dm', 'ppo',
 'roc', 'rocp', 'rocr', 'rocr100', 'rsi', 'slowk', 'slowd', 'fastk',
 'fastd', 'fastk_rsi', 'fastd_rsi', 'trix', 'ultosc', 'willr',
 'ad', 'adosc', 'obv', 'atr', 'natr', 'ht_dcperiod', 'ht_dcphase',
 'ht_phasor_inphase', 'ht_phasor_quadrature', 'ht_sine_sine', 'ht_sine_leadsine',
 'ht_trendmod', 'avgprice', 'medprice', 'typprice', 'wclprice']

In [16]:
TECHNICAL_PATTERNS = [g for g in df_full.keys() if g.find('cdl')>=0]
print(f'Technical patterns count = {len(TECHNICAL_PATTERNS)}, examples = {TECHNICAL_PATTERNS[0:5]}')

Technical patterns count = 61, examples = ['cdl2crows', 'cdl3blackrows', 'cdl3inside', 'cdl3linestrike', 'cdl3outside']


In [17]:
MACRO = ['gdppot_us_yoy', 'gdppot_us_qoq', 'cpi_core_yoy', 'cpi_core_mom', 'FEDFUNDS',
 'DGS1', 'DGS5', 'DGS10']

In [18]:
NUMERICAL = GROWTH + TECHNICAL_INDICATORS + TECHNICAL_PATTERNS + CUSTOM_NUMERICAL + MACRO

In [19]:
# CHECK: NO OTHER INDICATORS LEFT
OTHER = [k for k in df_full.keys() if k not in OHLCV + CATEGORICAL + NUMERICAL + TO_DROP]
OTHER

['growth_future_5d', 'is_positive_growth_5d_future']

In [20]:
# what are the categorical features?
CATEGORICAL

['Month', 'Weekday', 'Ticker', 'ticker_type']

In [21]:
df_full

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d,ln_volume
0,0.088542,0.101563,0.088542,0.097222,0.060163,1.031789e+09,MSFT,1986,1986-03-01,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.754560
1,0.097222,0.102431,0.097222,0.100694,0.062311,3.081600e+08,MSFT,1986,1986-03-01,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.546130
2,0.100694,0.103299,0.100694,0.102431,0.063386,1.331712e+08,MSFT,1986,1986-03-01,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.707146
3,0.102431,0.103299,0.098958,0.099826,0.061774,6.776640e+07,MSFT,1986,1986-03-01,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.031577
4,0.099826,0.100694,0.097222,0.098090,0.060700,4.789440e+07,MSFT,1986,1986-03-01,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.684509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5422,3639.000000,3648.949951,3584.050049,3594.300049,3594.300049,1.571996e+06,LT.NS,2024,2024-04-01,1,...,1.011164,1.108923,0.936075,0.949809,0.956129,0.913106,0.850046,1.423982,2.158543,14.267857
5423,3590.050049,3634.149902,3576.050049,3599.500000,3599.500000,3.748847e+06,LT.NS,2024,2024-05-01,3,...,0.973473,1.053911,0.931945,1.014925,0.926103,0.916902,0.903379,1.369046,2.038296,15.136959
5424,3610.000000,3622.000000,3488.449951,3499.800049,3499.800049,4.079696e+06,LT.NS,2024,2024-05-01,4,...,0.967125,1.049197,0.946816,1.063704,1.037155,0.986425,0.953153,1.462818,2.180063,15.221533
5425,3522.800049,3527.000000,3441.100098,3463.300049,3463.300049,2.614667e+06,LT.NS,2024,2024-05-01,0,...,0.975418,1.027877,0.952887,0.986426,1.004327,0.989362,0.916771,1.465996,2.219715,14.776647


In [27]:
# prompt: data['wom'] = data[0].apply(lambda d: (d.day-1) // 7 + 1)

df_full['wom'] = df_full['Month'].apply(lambda d: (d.day - 1) // 7 + 1)


In [28]:

# truncated df_full with 25 years of data (and defined growth variables)
df = df_full[df_full.Date>='2000-01-01']
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 182675 entries, 3490 to 5426
Columns: 204 entries, Open to wom
dtypes: datetime64[ns](3), float64(129), int32(64), int64(6), object(2)
memory usage: 241.1+ MB


In [29]:
# let look at the features count and size:
df[NUMERICAL].info()

<class 'pandas.core.frame.DataFrame'>
Index: 182675 entries, 3490 to 5426
Columns: 184 entries, growth_1d to DGS10
dtypes: float64(121), int32(62), int64(1)
memory usage: 214.6 MB


In [30]:
# dummy variables are not generated from Date and numeric variables
df.loc[:,'Month'] = df.Month.dt.strftime('%B')
df.loc[:,'Weekday'] = df.Weekday.astype(str)

In [31]:
df

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d,ln_volume,wom
3490,58.687500,59.312500,56.000000,58.281250,36.065567,53228400.0,MSFT,2000,January,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.790103,1
3491,56.781250,58.562500,56.125000,56.312500,34.847271,54119000.0,MSFT,2000,January,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.806696,1
3492,55.562500,58.187500,54.687500,56.906250,35.214706,64059600.0,MSFT,2000,January,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.975324,1
3493,56.093750,56.937500,54.187500,55.000000,34.035072,54976600.0,MSFT,2000,January,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.822418,1
3494,54.312500,56.125000,53.656250,55.718750,34.479843,62013600.0,MSFT,2000,January,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.942864,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5422,3639.000000,3648.949951,3584.050049,3594.300049,3594.300049,1571996.0,LT.NS,2024,April,1,...,1.108923,0.936075,0.949809,0.956129,0.913106,0.850046,1.423982,2.158543,14.267857,1
5423,3590.050049,3634.149902,3576.050049,3599.500000,3599.500000,3748847.0,LT.NS,2024,May,3,...,1.053911,0.931945,1.014925,0.926103,0.916902,0.903379,1.369046,2.038296,15.136959,1
5424,3610.000000,3622.000000,3488.449951,3499.800049,3499.800049,4079696.0,LT.NS,2024,May,4,...,1.049197,0.946816,1.063704,1.037155,0.986425,0.953153,1.462818,2.180063,15.221533,1
5425,3522.800049,3527.000000,3441.100098,3463.300049,3463.300049,2614667.0,LT.NS,2024,May,0,...,1.027877,0.952887,0.986426,1.004327,0.989362,0.916771,1.465996,2.219715,14.776647,1


In [32]:
# prompt: list wom unique

print(df['wom'].unique())


[1]


In [33]:
# Generate dummy variables (no need for bool, let's have int32 instead)
dummy_variables = pd.get_dummies(df[CATEGORICAL], dtype='int32')

In [34]:
# get dummies names in a list
DUMMIES = dummy_variables.keys().to_list()

In [35]:
# Concatenate the dummy variables with the original DataFrame
df_with_dummies = pd.concat([df, dummy_variables], axis=1)

In [36]:
df_with_dummies[NUMERICAL+DUMMIES].info()

<class 'pandas.core.frame.DataFrame'>
Index: 182675 entries, 3490 to 5426
Columns: 239 entries, growth_1d to ticker_type_US
dtypes: float64(121), int32(117), int64(1)
memory usage: 253.0 MB


In [37]:
df_with_dummies

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,Ticker_RMS.PA,Ticker_SAP,Ticker_SBIN.NS,Ticker_SIE.DE,Ticker_TCS.NS,Ticker_TTE,Ticker_V,ticker_type_EU,ticker_type_INDIA,ticker_type_US
3490,58.687500,59.312500,56.000000,58.281250,36.065567,53228400.0,MSFT,2000,January,0,...,0,0,0,0,0,0,0,0,0,1
3491,56.781250,58.562500,56.125000,56.312500,34.847271,54119000.0,MSFT,2000,January,1,...,0,0,0,0,0,0,0,0,0,1
3492,55.562500,58.187500,54.687500,56.906250,35.214706,64059600.0,MSFT,2000,January,2,...,0,0,0,0,0,0,0,0,0,1
3493,56.093750,56.937500,54.187500,55.000000,34.035072,54976600.0,MSFT,2000,January,3,...,0,0,0,0,0,0,0,0,0,1
3494,54.312500,56.125000,53.656250,55.718750,34.479843,62013600.0,MSFT,2000,January,4,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5422,3639.000000,3648.949951,3584.050049,3594.300049,3594.300049,1571996.0,LT.NS,2024,April,1,...,0,0,0,0,0,0,0,0,1,0
5423,3590.050049,3634.149902,3576.050049,3599.500000,3599.500000,3748847.0,LT.NS,2024,May,3,...,0,0,0,0,0,0,0,0,1,0
5424,3610.000000,3622.000000,3488.449951,3499.800049,3499.800049,4079696.0,LT.NS,2024,May,4,...,0,0,0,0,0,0,0,0,1,0
5425,3522.800049,3527.000000,3441.100098,3463.300049,3463.300049,2614667.0,LT.NS,2024,May,0,...,0,0,0,0,0,0,0,0,1,0


In [38]:
corr_is_positive_growth_5d_future = df_with_dummies[NUMERICAL+DUMMIES+TO_PREDICT].corr()['is_positive_growth_5d_future']